# Produce IO Tables from OECD data

This notebook generalises the code from the notebook "WS3_ACC_Produce_OECD_IO_tables_AUS__tinker" to multiple countries in one year. Its aim is to produce Input-Output (IO) tables for a number of countries using the OECD data published [here](https://www.oecd.org/sti/ind/inter-country-input-output-tables.htm). At the moment, its output for each country is a 36x36 matrix of intermediate consumption by sectors, which is extended with three additional rows accounting for imports, value added and total production by sector. Hence, the final output's dimenssions are 39x36.

In [1]:
# Imports and path
import pymrio
import pandas as pd
from pathlib import Path
oecd_storage = Path('/project_data/data_asset/')

In [2]:
# Select the year to parse - 2015 in this case
oecd_path_year = pymrio.parse_oecd(path=oecd_storage, year=2015)

In [3]:
# Show countries
print('There are', len(oecd_path_year.get_regions()), 'countries in the dataset:')
print(set(oecd_path_year.get_regions()))

There are 65 countries in the dataset:
{'MAR', 'KHM', 'CRI', 'DNK', 'ESP', 'BEL', 'KOR', 'MLT', 'SVK', 'DEU', 'POL', 'TUN', 'BGR', 'ROW', 'IRL', 'AUS', 'SAU', 'EST', 'HUN', 'CHE', 'ITA', 'JPN', 'PER', 'RUS', 'IDN', 'FIN', 'COL', 'LUX', 'GRC', 'NOR', 'MYS', 'GBR', 'ZAF', 'KAZ', 'ISR', 'LVA', 'NLD', 'SGP', 'BRN', 'CYP', 'HKG', 'NZL', 'SWE', 'SVN', 'MEX', 'THA', 'PRT', 'CZE', 'TUR', 'AUT', 'TWN', 'USA', 'BRA', 'VNM', 'ISL', 'ROU', 'LTU', 'CAN', 'ARG', 'CHN', 'FRA', 'PHL', 'CHL', 'IND', 'HRV'}


In [4]:
# Let's work with a subsample for the moment: UK, Spain, Italy, France, Germany, Belgium and Singapore
countries = ['GBR', 'ESP', 'ITA', 'FRA', 'DEU', 'BEL', 'SGP']

In [5]:
# Constrcut Input-Output tables for selected countries
for country in countries:
    
    print("Now processing:", country)
    
    # INTERMEDIATE DOMESTIC CONSUMPTION
    # Extract domestic intermmediate consumption
    IO = oecd_path_year.Z.iloc[oecd_path_year.Z.index.get_level_values('region') == country, 
                               oecd_path_year.Z.columns.get_level_values('region') == country]
    
    IO.reset_index(level = 0, drop = True, inplace = True)
    IO.columns = IO.columns.droplevel()
    
    check = IO.index == IO.columns 
    assert check.all() == True
    
    
    # IMPORTS
    # Extract all entries that are not from domestic sectors feeding into domestic columns
    IMP = oecd_path_year.Z.iloc[oecd_path_year.Z.index.get_level_values('region') != country,     # Non-Domestic rows
                               oecd_path_year.Z.columns.get_level_values('region') == country]    # Domestic columns
    
    
    # Create a new dataframe that combines intermediate consumption (IO) with total imports (IMP), called IO_EXT
    # The table is no longer symmetric
    IO_EXT = IO.copy()
    IO_EXT.loc['IMPORTS'] = pd.Series(IMP.sum()).values
    
    
    # TAXES AND VALUE ADDED
    # Break down of taxes and subsidies by sector just for the country
    TAXSUB = oecd_path_year.factor_inputs.F.iloc[oecd_path_year.factor_inputs.F.index.get_level_values('inputtype') == (country + '_TAXSUB'),
                                                   oecd_path_year.factor_inputs.F.columns.get_level_values('region') == country]
    TAXSUB.columns = TAXSUB.columns.droplevel()
    
    # Overseas taxes and subsidies
    INT_TAXSUB = oecd_path_year.factor_inputs.F.iloc[oecd_path_year.factor_inputs.F.index.get_level_values('inputtype') != (country + '_TAXSUB'), # select all non-domestic rows 
                                                     oecd_path_year.factor_inputs.F.columns.get_level_values('region') == country]                # select only domestic columns
    INT_TAXSUB.columns = INT_TAXSUB.columns.droplevel()
    INT_TAXSUB = INT_TAXSUB[INT_TAXSUB.index != 'VALU'] # Drop value added row
    
    # Value added of Australian sectors
    VA = oecd_path_year.factor_inputs.F.iloc[oecd_path_year.factor_inputs.F.index.get_level_values('inputtype') == 'VALU',
                                                oecd_path_year.factor_inputs.F.columns.get_level_values('region') == country]
    VA.columns = VA.columns.droplevel()
    
    # Add Value Added and Taxes to IO_EXT
    VA_TAXES = VA.append(TAXSUB * (-1))
    VA_TAXES.loc['INT_TAXSUB'] = pd.Series(INT_TAXSUB.sum()).values * (-1)

    IO_EXT.loc['VALUE ADDED'] = pd.Series(VA_TAXES.sum()).values
    
    
    # TOTAL PRODUCTION
    IO_EXT.loc['TOTAL PRODUCTION'] = pd.Series(IO_EXT.sum()).values
    
    
    # EXPORT TO CSV
    IO_EXT.to_csv('/project_data/data_asset/IO_'+ country + '.csv')

print('Done!')

Now processing: GBR
Now processing: ESP
Now processing: ITA
Now processing: FRA
Now processing: DEU
Now processing: BEL
Now processing: SGP
Done!


In [8]:
test = pd.read_csv('/project_data/data_asset/IO_GBR.csv')
test

,sector,01T03,05T06,07T08,09,10T12,13T15,16,17T18,19,...,61,62T63,64T66,68,69T82,84,85,86T88,90T96,97T98
0,01T03,3492.194837,12.842469,2.791720,3.220359,13729.756313,96.891339,255.340420,14.872379,2.397811,...,13.961189,17.623030,55.309668,9.880991,76.781178,37.064063,47.630506,67.265742,24.717992,0.000000
1,05T06,15.349830,508.254648,12.901800,75.645359,184.868985,9.772478,8.753637,35.031400,3828.700868,...,24.103292,20.681480,74.822725,100.201781,231.329085,86.461217,76.943649,244.476908,117.519485,0.000000
2,07T08,40.052759,2.669115,325.220727,23.220018,64.917748,1.843805,2.082504,44.522848,2.545895,...,3.615089,2.028899,6.533083,138.714080,65.131695,102.023397,15.832128,14.774671,26.966879,0.000000
3,09,220.448319,1806.022159,248.017319,286.170908,75.688963,0.000000,3.373589,19.011024,14.391949,...,0.000000,0.000000,1.441953,0.000000,65.593172,70.135361,25.325904,2.595770,0.662038,0.000000
4,10T12,1764.461574,19.130757,8.907392,3.817720,12297.637294,54.403320,19.063672,111.850025,17.775255,...,34.309072,36.289702,134.386814,80.340436,560.160826,769.453709,379.277569,1263.584162,980.226818,0.000000
5,13T15,17.639560,5.788987,3.100258,1.647468,65.240682,435.103301,8.344102,49.149909,1.692847,...,17.859468,13.063929,44.222737,8.969556,70.368752,81.870369,36.236406,92.630452,53.245958,0.000000
6,16,26.437661,41.726579,17.058281,6.284740,111.029437,21.569424,2245.519815,79.825142,0.814337,...,22.320979,14.879163,31.218657,28.986269,152.431567,26.025563,136.594860,30.814846,42.141324,0.000000
7,17T18,312.167874,17.901077,34.117786,4.926261,1801.783026,116.221532,114.311914,5299.588133,19.673268,...,241.340123,261.038759,2881.663227,301.715156,1334.031337,1956.787656,1669.491180,2111.699885,551.936205,0.000000
8,19,392.356951,132.044402,86.892930,30.061135,214.583449,27.996972,31.028444,37.159286,593.053314,...,139.216290,86.169392,254.157627,19.746650,418.348592,355.797115,124.568384,149.692805,108.483115,0.000000
9,20T21,491.171381,145.803922,73.256178,23.144042,798.789047,432.889859,104.320850,624.732311,98.873590,...,108.649890,99.862646,257.001397,86.956889,924.035164,329.796474,334.669542,1170.089637,314.865244,0.000000
